In [22]:
import polars as pl
import re
import emoji

def cleaner(text):
    text = str(text).lower() # Set all the words in lower case
    text = re.sub(r'\n', ' ', text) # Remove newline characters
    text = re.sub("@[A-Za-z0-9]+","",text) # Remove usernames
    text = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", text) # Remove the URLs
    text = " ".join(text.split()) # Remove extra white spaces
    text = ''.join(c for c in text if c not in emoji.EMOJI_DATA) # Remove emojis
    text = text.replace("#", "").replace("_", " ")
    text = text.replace("http", "")
    text = re.sub(r"[^\w\s]", "", text)  # Remove punctuation
    text = re.sub(r"\d+", "", text)  # Remove numbers
    text = re.sub(r"\s+", " ", text).strip()  # Remove extra whitespaces
    
    return text

def encode_label(label):
    return 0 if label == "NOT" else 1

df1 = pl.read_csv('OLID/testset-levela.tsv', separator="\t")
df2 = pl.read_csv('OLID/labels-levela.csv')
df = df1.join(df2, on="id").with_columns(
    text = pl.col("tweet").map_elements(cleaner, return_dtype=pl.Utf8),
    labels = pl.col("label").map_elements(encode_label, return_dtype=pl.Int64)
)

df.select(["text", "labels"]).write_csv("datasets/dataset4_test.csv")

In [23]:
df = pl.read_csv("OLID/olid-training-v1.0.tsv", separator="\t").with_columns(
    text = pl.col("tweet").map_elements(cleaner, return_dtype=pl.Utf8),
    labels = pl.col("subtask_a").map_elements(encode_label, return_dtype=pl.Int64)
)

df.select(["text", "labels"]).write_csv("datasets/dataset4_train.csv")